# Análise de dados - Netflix 

Essa análise de dados foi feita utilizando um conjunto de dados coletado do Flixable, um mecanismo de pesquisa de terceiros da Netflix.

---
# Dados

- **'show_id'**: ID único para cada filme / programa de TV.
- **'type'**: se é filme ou programa de TV.
- **'title'**: Título do filme / programa de TV.
- **'director'**: Diretor do filme/ programa de TV.
- **'cast'**: Atores envolvidos no filme/ programa de TV.
- **'country'**: País onde o filme/ programa de TV foi produzido.
- **'date_added'**: Data em que foi adicionado ao Netflix.
- **'release_year'**: Ano de lançamento real do filme/ programa de TV.
- **'rating'**: Classificação do filme/ programa de TV.
- **'duration'**: Duração total - em minutos ou em números de temporadas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from pandas_profiling import ProfileReport
from scipy.stats import norm
from sklearn.preprocessing import MultiLabelBinarizer
from wordcloud import WordCloud

In [ ]:
df = pd.read_csv('../input/netflix-data/netflix_titles.csv')
df.head()

## df.info()

In [ ]:
report = ProfileReport(df)
report

# Limpeza de Dados

In [ ]:
df.nunique()

In [ ]:
df.isna().sum()

**director** e **cast**: não são necessárias para a análise (sem muita informação).

**country**: é uma variável importante,então será necessário corrigi-lá. 

**date_added** : Não foi possível achar os valores para esta coluna na internet, então as linhas que não contêm o valor serão descartadas, pois são apenas 11.

**rating** : os dados para esta coluna pode ser encontrados na Amazon ranting.

### Rating

In [ ]:
df[df['rating'].isna()]

In [ ]:
rating_replacements = {
    67: 'TV-PG',
    2359: 'TV-14',
    3660: 'TV-MA',
    3736: 'TV-MA',
    3737: 'NR',
    3738: 'TV-MA',
    4323: 'TV-MA '
}

for id, rate in rating_replacements.items():
    df.iloc[id, 8] = rate

df['rating'].isna().sum()

### Director e Cast

In [ ]:
df = df.drop(['director', 'cast'], axis=1)

### date_added 

In [ ]:
df = df[df['date_added'].notna()]

### Country
-  os valores nulos serão trocados pelo país com maior numero de repetições na coluna.

In [ ]:
df['country'] = df['country'].fillna(df['country'].mode()[0])

##  Verificando se os dados estão limpos

In [ ]:
df.isna().sum()

# Cirando novas colunas com base em variáveis

### Datas
- Pra uma melhor visualização das datas elas sarão separadas em 3 colunas:
ano_adicionado, mes_adicionado e trimestre_adicionado.

In [ ]:
df['ano_adicionado'] = pd.DatetimeIndex(df['date_added']).year
df['mes_adicionado'] = pd.DatetimeIndex(df['date_added']).month
df['trimestre_adicionado'] = pd.DatetimeIndex(df['date_added']).quarter

### Público alvo
- o público alvo será definido com base na Classificação Indicativa (rating).

In [ ]:
classificacao_indicativa = {
    'TV-PG': 'Crianças mais velhas',
    'TV-MA': 'Adultos',
    'TV-MA ': 'Adultos',
    'TV-Y7-FV': 'Crianças mais velhas',
    'TV-Y7': 'Crianças mais velhas',
    'TV-14': 'Adolescentes',
    'R': 'Adultos',
    'TV-Y': 'Crianças',
    'NR': 'Adultos',
    'PG-13': 'Adolescentes',
    'TV-G': 'Crianças',
    'PG': 'Crianças mais velhas',
    'G': 'Crianças',
    'UR': 'Adultos',
    'NC-17': 'Adultos'
}

df['publico_alvo'] = df['rating'].replace(classificacao_indicativa)
df['publico_alvo'].unique()

### Country
- Em algumas linhas a coluna country contém uma lista com varios países. Para facilitar, será usado apenas o primeiro país de cada lista.

In [ ]:
df['pais_principal'] = df['country'].apply(lambda x: x.split(",")[0])

### gênero
- a coluna listed_in tem os gêneros que pertencem aos filmes / programas de tv , porem eles estão juntos em apenas uma string, impossibilitando a leitura dos dados. Para que seja possível uma leitura de dados adequada é necessário separar todos os gêneros em uma nova coluna.

In [ ]:
df['genero'] = df['listed_in'].apply(lambda x :  x.replace(' ,',',').replace(', ',',').split(',')) 

# Ajustando os dados

### Type e publico_alvo
- As colunas type e publico_alvo devem ser uma categoria.

In [ ]:
df['type'] = pd.Categorical(df['type'])

df['publico_alvo'] = pd.Categorical(df['publico_alvo'], 
                                   categories=['Crianças', 'Crianças mais velhas', 
                                               'Adolescentes', 'adultos'])

### Separando filmes e programas de TV

In [ ]:
df['tipo'] = df['type'].replace('TV Show','Programa de TV').replace('Movie','Filme')

filme_df = df[df['tipo'] == 'Filme']
programa_df = df[df['tipo'] == 'Programa de TV']

# Retirando colunas desnecessárias

In [ ]:
df = df.drop(['type', 'date_added', 'listed_in','country','description'], axis=1)

In [ ]:
df.head()

# visualização dos dados

In [ ]:
fig = px.pie(df['tipo'].value_counts().reset_index(), values='tipo', names='index')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
def generate_rating_df(df):
    rating_df = df.groupby(['rating', 'publico_alvo']).agg({'show_id': 'count'}).reset_index()
    rating_df = rating_df[rating_df['show_id'] != 0]
    rating_df.columns = ['classificacao', 'publico_alvo', 'quantidade']
    rating_df = rating_df.sort_values('publico_alvo')
    return rating_df

In [ ]:
rating_df = generate_rating_df(df)
fig = px.bar(rating_df, x='classificacao', y='quantidade', color='publico_alvo')
fig.show()

In [ ]:
filme_classif_df = generate_rating_df(filme_df)
programa_classif_df = generate_rating_df(programa_df)

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(
    go.Pie(labels=filme_classif_df['publico_alvo'], values=filme_classif_df['quantidade']),
    row=1, col=1
)

fig.add_trace(
    go.Pie(labels=programa_classif_df['publico_alvo'], values=programa_classif_df['quantidade']),
    row=1, col=2
)

fig.update_traces(textposition='inside', hole=.4, hoverinfo="label+percent")
fig.update_layout(
    title_text="Distribuição de classificação por tipo de conteúdo",
    annotations=[dict(text='     Filmes', x=0.16, y=0.5, font_size=12, showarrow=False),
                 dict(text='Programas', x=0.82, y=0.5, font_size=12, showarrow=False)])
fig.show()

In [ ]:
country_df = df['pais_principal'].value_counts().reset_index()
country_df = country_df[country_df['pais_principal'] /  country_df['pais_principal'].sum() > 0.01]

fig = px.pie(country_df, values='pais_principal', names='index')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.histogram(df, x='pais_principal')
fig.update_xaxes(categoryorder='total descending')
fig.show()

## Distribuição do conteúdo

In [ ]:
ano_lancado_df = df.loc[df['release_year'] > 2010].groupby(['release_year', 'tipo']).agg({'show_id': 'count'}).reset_index()
ano_adicionado_df = df.loc[df['ano_adicionado'] > 2010].groupby(['ano_adicionado', 'tipo']).agg({'show_id': 'count'}).reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter( 
    x=ano_lancado_df.loc[ano_lancado_df['tipo'] == 'Filme']['release_year'], 
    y=ano_lancado_df.loc[ano_lancado_df['tipo'] == 'Filme']['show_id'],
    mode='lines+markers',
    name='filme: ano de lançamento',
    marker_color='green',
))
fig.add_trace(go.Scatter( 
    x=ano_lancado_df.loc[ano_lancado_df['tipo'] == 'Programa de TV']['release_year'], 
    y=ano_lancado_df.loc[ano_lancado_df['tipo'] == 'Programa de TV']['show_id'],
    mode='lines+markers',
    name='Programa de TV: ano de lançamento',
    marker_color='darkgreen',
))
fig.add_trace(go.Scatter( 
    x=ano_adicionado_df.loc[ano_adicionado_df['tipo'] == 'Filme']['ano_adicionado'], 
    y=ano_adicionado_df.loc[ano_adicionado_df['tipo'] == 'Filme']['show_id'],
    mode='lines+markers',
    name='filme: ano de adicionado',
    marker_color='orange',
))
fig.add_trace(go.Scatter( 
    x=ano_adicionado_df.loc[ano_adicionado_df['tipo'] == 'Programa de TV']['ano_adicionado'], 
    y=ano_adicionado_df.loc[ano_adicionado_df['tipo'] == 'Programa de TV']['show_id'],
    mode='lines+markers',
    name='Programa de TV: ano adicionado',
    marker_color='darkorange',
))
fig.update_xaxes(categoryorder='total descending')
fig.show()

## Análise de gênero

In [ ]:
def calculate_mlb(series):
    mlb = MultiLabelBinarizer()
    mlb_df = pd.DataFrame(mlb.fit_transform(series), columns=mlb.classes_, index=series.index)
    return mlb_df

In [ ]:
def top_genres(df, title='Melhores'):
    genero_df = calculate_mlb(df['genero'])
    tdata = genero_df.sum().sort_values(ascending=False)
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=tdata.index,
        y=tdata.values,
    ))
    fig.update_layout(title=title)
    fig.show()

#### filmes

In [ ]:
top_genres(filme_df, title='genêros mais famosos (filmes)')

In [ ]:
filme_top_generos =  [
    'International Movies',
    'Dramas',
    'Comedies',
    'Documentaries',
    'Action & Adventure',
]

#### Programas

In [ ]:
top_genres(programa_df, title='genêros mais famosos (programas)')

In [ ]:
text = str(list(filme_df['genero'])).replace(',', '').replace('[', '').replace("'", '').replace(']', '')

plt.rcParams['figure.figsize'] = (15, 15)
wordcloud = WordCloud(background_color = 'white', width = 1200,  height = 1200, max_words = 121).generate(text)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

## Análise de duração

### Programas de TV

In [ ]:
fig = px.histogram(x=programa_df['duration'])
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(
    title="Distribuição da duração dos programas",
    xaxis_title="Duração do programa",
)
fig.show()

### Filmes

 a duração dos filmes será dividida em três categorias:

   - Menos de 1.30 horas.
   - Mais de 1.30 horas e menos de 2.30 horas.
   - Mais de 2.30 horas.

In [ ]:
temp_duracao = filme_df['duration'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
filme_df.loc[temp_duracao.loc[temp_duracao < 90].index, 'duration_bin'] = 'Menos de 1.30 horas'
filme_df.loc[temp_duracao.loc[(temp_duracao >= 90) & (temp_duracao < 150)].index, 'duration_bin'] = 'Mais de 1.30 horas e menos de 2.30 horas'
filme_df.loc[temp_duracao.loc[temp_duracao >= 150].index, 'duration_bin'] = 'Mais de 2.30 horas'

In [ ]:
fig = px.histogram(x=filme_df['duration_bin'], color=filme_df['duration_bin'])
fig.update_layout(
    title="Distribuição da duração dos filmes",
    xaxis_title="Duração dos filmes",
)
fig.show()

## Lançamento do ano por gêneros

In [ ]:
filme_df['genero_principal'] = filme_df['genero'].apply(lambda genres: genres[0])

In [ ]:
genero_ano_df = filme_df[(filme_df['genero_principal'].isin(filme_top_generos)) & (filme_df['ano_adicionado'] >= 2017)].groupby(['genero_principal', 'ano_adicionado']).agg({'title': 'count'})
genero_ano_df = genero_ano_df.reset_index()
genero_ano_df.columns = ['genero_principal', 'ano_adicionado', 'quantidade']

fig = px.sunburst(genero_ano_df, path=['ano_adicionado', 'genero_principal'], values='quantidade')
fig.show()